In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from optimum_benchmark import Benchmark, BenchmarkConfig, TorchrunConfig, InferenceConfig, PyTorchConfig
from optimum_benchmark.logging_utils import setup_logging
from codecarbon import EmissionsTracker  # energy consumption tracking
import json

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
backend = "pytorch"
    
    
def run_benchmark(task_type: str, model_name=model_name, backend=backend):

    setup_logging(level="INFO")
    
    # Adjust input configuration based on task type
    if task_type == "reasoning":
        # Reasoning tasks might need longer chain-of-thought prompts or a certain structure
        input_shapes = {"sequence_length": 256}  # example value, adjust as needed
    elif task_type == "summarisation":
        # Summarisation tasks often process long text inputs
        input_shapes = {"sequence_length": 512}
    elif task_type == "generative":
        # Open-ended generative tasks may have varied lengths
        input_shapes = {"sequence_length": 128}
    else:
        raise ValueError("Unsupported task type provided.")

    launcher_config = TorchrunConfig(nproc_per_node=2)
    
    scenario_config = InferenceConfig(
        latency=True, 
        memory=True, 
        input_shapes=input_shapes
    )
    
    backend_config = PyTorchConfig(
        model=model_name, 
        device="cuda", 
        device_ids="0,1", 
        no_weights=True
    )
    
    benchmark_config = BenchmarkConfig(
        name=f"{backend}_{model_name}_{task_type}",
        scenario=scenario_config,
        launcher=launcher_config,
        backend=backend_config,
    )
    
    # Launch the benchmark and capture the report
    benchmark_report = Benchmark.launch(benchmark_config)
    benchmark_results = benchmark_report.to_dict()
    
    return benchmark_results

def run_energy_tracked_benchmark(task_type: str):
    """
    Wrap the benchmark run with an energy consumption tracker.
    """
    tracker = EmissionsTracker()
    tracker.start()
    
    results = run_benchmark(task_type)
    
    # Stop the tracker and get the energy consumption metrics
    emissions: float = tracker.stop()
    results["energy_consumption_kWh"] = emissions
    return results


for task in ["reasoning", "summarisation", "generative"]:
    print(f"Running benchmark for {task} task:")
    task_results = run_energy_tracked_benchmark(task)
    print(json.dumps(task_results, indent=2))

[codecarbon INFO @ 15:34:44] [setup] RAM Tracking...
[codecarbon INFO @ 15:34:44] [setup] CPU Tracking...
[codecarbon WARNING @ 15:34:44] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU



Running benchmark for reasoning task:


[codecarbon INFO @ 15:34:45] CPU Model on constant consumption mode: AMD EPYC 7742 64-Core Processor
[codecarbon INFO @ 15:34:45] [setup] GPU Tracking...
[codecarbon INFO @ 15:34:45] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:34:45] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 15:34:45] GPU number 3 will not be monitored, at your request.
[codecarbon WARNING @ 15:34:45] You have 4 GPUs but we will monitor only 2 of them. Check your configuration.
[codecarbon INFO @ 15:34:46] >>> Tracker's metadata:
[codecarbon INFO @ 15:34:46]   Platform system: Linux-5.15.0-113-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 15:34:46]   Python version: 3.10.14
[codecarbon INFO @ 15:34:46]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 15:34:46]   Available RAM : 503.532 GB
[codecarbon INFO @ 15:34:46]   CPU count: 128
[codecarbon INFO @ 15:34:46]   CPU model: AMD EPYC 7742 64-Core Processor
[codecarbon INFO @ 15:34:46]   GPU count: 2
[codecarbon INFO @ 15:34:46]  

[2025-03-14 15:34:50,363][backend][INFO] - CUDA_VISIBLE_DEVICES was set to 0,1.
[2025-03-14 15:34:50,443][torchrun][INFO] - Allocated torchrun launcher


[codecarbon INFO @ 15:35:04] Energy consumed for RAM : 0.000788 kWh. RAM Power : 188.8243260383606 W
[codecarbon INFO @ 15:35:04] Energy consumed for all CPUs : 0.000470 kWh. Total CPU Power : 112.5 W
[codecarbon INFO @ 15:35:04] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 15:35:04] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 15:35:04] Energy consumed for all GPUs : 0.000351 kWh. Total GPU Power : 83.96057158256603 W
[codecarbon INFO @ 15:35:04] 0.001608 kWh of electricity used since the beginning.
[W socket.cpp:663] [c10d] The client socket has failed to connect to [ds01]:37575 (errno: 22 - Invalid argument).
[W socket.cpp:663] [c10d] The client socket has failed to connect to [ds01]:37575 (errno: 22 - Invalid argument).
[W socket.cpp:663] [c10d] The client socket has failed to connect to [ds01]:37575 (errno: 22 - Invalid argument).
[W socket.cpp:663] [c10d] The client socket has failed to connect to [ds01]:37575 (errno: 22 - Inv

...

[RANK-PROCESS-0][2025-03-14 15:38:34,954][inference][INFO] - 	+ Running Per-Token Text Generation latency tracking


[codecarbon INFO @ 15:38:43] Energy consumed for RAM : 0.004658 kWh. RAM Power : 188.8243260383606 W
[codecarbon INFO @ 15:38:43] Energy consumed for all CPUs : 0.002777 kWh. Total CPU Power : 112.5 W
[codecarbon INFO @ 15:38:43] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 15:38:43] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 15:38:43] Energy consumed for all GPUs : 0.002246 kWh. Total GPU Power : 146.0110890194854 W
[codecarbon INFO @ 15:38:43] 0.009681 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:38:58] Energy consumed for RAM : 0.005441 kWh. RAM Power : 188.8243260383606 W
[codecarbon INFO @ 15:38:58] Energy consumed for all CPUs : 0.003243 kWh. Total CPU Power : 112.5 W
[codecarbon INFO @ 15:38:58] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 15:38:58] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 15:38:58] Energy consumed for all GPUs : 0.002759 kWh. T

[RANK-PROCESS-0][2025-03-14 15:39:38,166][inference][INFO] - 	+ Running Text Generation memory tracking


[codecarbon INFO @ 15:39:43] Energy consumed for RAM : 0.007794 kWh. RAM Power : 188.8243260383606 W
[codecarbon INFO @ 15:39:43] Energy consumed for all CPUs : 0.004646 kWh. Total CPU Power : 112.5 W
[codecarbon INFO @ 15:39:43] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 15:39:43] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 15:39:43] Energy consumed for all GPUs : 0.004288 kWh. Total GPU Power : 100.8928412773797 W
[codecarbon INFO @ 15:39:43] 0.016727 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:39:58] Energy consumed for RAM : 0.008579 kWh. RAM Power : 188.8243260383606 W
[codecarbon INFO @ 15:39:58] Energy consumed for all CPUs : 0.005114 kWh. Total CPU Power : 112.5 W
[codecarbon INFO @ 15:39:58] GPU number 2 will not be monitored, at your request.
[codecarbon INFO @ 15:39:58] GPU number 3 will not be monitored, at your request.
[codecarbon INFO @ 15:39:59] Energy consumed for all GPUs : 0.004643 kWh. T

: 

In [3]:
def display_dict(d):
    print(json.dumps(d, indent=4))
    
results = run_benchmark()

display_dict(results)

[2025-03-14 15:26:34,523][backend][INFO] - CUDA_VISIBLE_DEVICES was set to 0,1.
[2025-03-14 15:26:34,991][torchrun][INFO] - Allocated torchrun launcher


[2025-03-14 15:26:36,336] torch.distributed.elastic.multiprocessing.api: [WARNING] Unable to shutdown process 3145786 via Signals.SIGINT, forcefully exiting via Signals.SIGKILL


[ISOLATED-PROCESS][2025-03-14 15:26:46,615][torchrun][ERROR] - 	+ Sending traceback to main process
[ISOLATED-PROCESS][2025-03-14 15:26:46,625][torchrun][INFO] - 	+ Exiting isolated process
[RANK-PROCESS-0][2025-03-14 15:27:15,481][torchrun][INFO] - 	+ Setting torch.distributed cuda device to 0
[RANK-PROCESS-0][2025-03-14 15:27:15,591][torchrun][INFO] - 	+ Initializing torch.distributed process group
[RANK-PROCESS-0][2025-03-14 15:27:17,238][datasets][INFO] - PyTorch version 2.1.2 available.
[RANK-PROCESS-0][2025-03-14 15:27:20,021][pytorch][INFO] - Allocating pytorch backend
[RANK-PROCESS-0][2025-03-14 15:27:20,022][pytorch][INFO] - 	+ Seeding backend with 42
[RANK-PROCESS-0][2025-03-14 15:27:20,023][pytorch][INFO] - 	+ Benchmarking a Transformers model
[RANK-PROCESS-0][2025-03-14 15:27:21,275][inference][INFO] - Allocating inference scenario
[RANK-PROCESS-0][2025-03-14 15:27:21,277][inference][INFO] - 	+ Updating Text Generation kwargs with default values
[RANK-PROCESS-0][2025-03-14 

[2025-03-14 15:29:05,409] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 3147208 via signal SIGTERM


[ISOLATED-PROCESS][2025-03-14 15:29:11,049][torchrun][INFO] - 	+ Sending outputs to main process
[ISOLATED-PROCESS][2025-03-14 15:29:11,050][torchrun][INFO] - 	+ Exiting isolated process
[2025-03-14 15:29:12,930][torchrun][INFO] - 	+ Received report from rank process [0]
[2025-03-14 15:29:12,946][torchrun][INFO] - 	+ Received report from rank process [1]
[2025-03-14 15:29:12,957][torchrun][INFO] - 	+ Aggregating reports from all rank processes
[2025-03-14 15:29:13,053][report][INFO] - + load_model:
[2025-03-14 15:29:13,060][report][INFO] - 	+ memory:
[2025-03-14 15:29:13,062][report][INFO] - 		+ max_ram: 1682.73 (MB)
[2025-03-14 15:29:13,065][report][INFO] - 		+ max_global_vram: 27570.34 (MB)
[2025-03-14 15:29:13,065][report][INFO] - 		+ max_process_vram: 9831.45 (MB)
[2025-03-14 15:29:13,066][report][INFO] - 		+ max_reserved: 8803.84 (MB)
[2025-03-14 15:29:13,067][report][INFO] - 		+ max_allocated: 8800.39 (MB)
[2025-03-14 15:29:13,068][report][INFO] - 	+ latency:
[2025-03-14 15:29:13